<a href="https://colab.research.google.com/github/alexkabu-M2K/Colab_Google_Estacio/blob/main/speech_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import IPython
import pyaudio
import wave
import speech_recognition as sr

In [ ]:
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RECORD_SECONDS = 10
WAVE_OUTPUT_FILENAME = "output.wav"

p = pyaudio.PyAudio()

In [ ]:
stream = p.open(
    format=FORMAT,
    rate=RATE,
    input = True,
    frames_per_buffer=CHUNK,
    channels=CHANNELS
)

print("* recording")
frames=[]

for i in range(0, int( RATE/CHUNK*RECORD_SECONDS )):
    data = stream.read(CHUNK)
    frames.append(data)

print("* Done recording...")
stream.stop_stream()
stream.close()
p.terminate()

wf = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
wf.setnchannels(CHANNELS)
wf.setsampwidth(p.get_sample_size(FORMAT))
wf.setframerate(RATE)
wf.writeframes(b''.join(frames))
wf.close()

* recording
* Done recording...


In [ ]:
print("Reproducing Audio:")
IPython.display.Audio(WAVE_OUTPUT_FILENAME)

Reproducing Audio:


In [ ]:
r = sr.Recognizer()

In [ ]:
with sr.AudioFile(WAVE_OUTPUT_FILENAME) as source:
    read_audio = r.record(source)

In [ ]:
transcribed_audio = r.recognize_google(
    audio_data=read_audio,
    language='pt-BR'
)

result2:
{   'alternative': [   {   'confidence': 0.82141507,
                           'transcript': 'o rato roeu a roupa do rei de Roma'}],
    'final': True}


In [ ]:
print("Transcription: ", transcribed_audio)

Transcription:  o rato roeu a roupa do rei de Roma


In [ ]:
import nltk
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('mac_morpho')
from nltk.corpus import stopwords

stop_words = set(stopwords.words("portuguese"))


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fernandodurier/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fernandodurier/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package mac_morpho to
[nltk_data]     /Users/fernandodurier/nltk_data...
[nltk_data]   Package mac_morpho is already up-to-date!


In [ ]:
from nltk.corpus import mac_morpho

def simplify_tag(t):
    if "+" in t:
        return t[t.index("+")+1:]
    else:
        return t



In [ ]:
tsents = mac_morpho.tagged_sents()
tsents = [[(w.lower(),simplify_tag(t)) for (w,t) in sent] for sent in tsents if sent]
train = tsents[100:]
test = tsents[:100]

tagger0 = nltk.DefaultTagger('n')
tagger1 = nltk.UnigramTagger(train, backoff=tagger0)
tagger2 = nltk.BigramTagger(train, backoff=tagger1)


In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize

def sentence_tokenization(text):
    try:
        return sent_tokenize(text)
    except Exception as e:
        print("Sent Tokenization Error: ", e)
        return []

def word_tokenization(text):
    try:
        return [token for token in word_tokenize(text) if token not in stop_words]
    except Exception as e:
        print("Word Tokenization Error: ", e)
        return []


In [ ]:
def word_pos_tag(word_tokens):
    pts = tagger1.tag(word_tokens)
    return pts


In [ ]:
word_tokens = word_tokenization( transcribed_audio.lower() )

In [ ]:
word_tokens

['rato', 'roeu', 'roupa', 'rei', 'roma']

In [ ]:
word_pos_tag(word_tokens)

[('rato', 'N'), ('roeu', 'n'), ('roupa', 'N'), ('rei', 'N'), ('roma', 'NPROP')]